# Imports

In [2]:
import spacy
import pandas as pd
import matplotlib.pyplot as mltp
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import sklearn as sk
import seaborn as sea
import re 
from tqdm import tqdm

# processing

In [12]:
filename='data/AFD_Wahlprogramm_2021.txt'

In [22]:
with open(filename) as f:
    text = f.read()
    #text = text.replace(r'\\d+\\n', '$')
    #text=   re.sub("\n\n", ".", text)
    #text=   re.sub("\d+.", ".", text)
    text=   re.sub(" \d+\n", ".", text)
    text=   re.sub("\n\d+", " ", text)
    text=   re.sub("\n", " ", text)
sentences = re.split(r' *[\.\?!][\'"\)\]]* *', text)

In [23]:
text=sentences

In [24]:
len(text)

2086

In [25]:
text[::-1]

['de  ',
 'de/spenden Paypal@alternativefuer',
 'afd',
 'AfD-ID  www',
 'Bankverbindung: Alternative für Deutschland Berliner Volksbank  IBAN: DE94 1loloicKofofeie, MTSWI:PIR) 940) :1(6% 1231477019) 3:7:)  Bitte geben Sie im Betreff mit an: Spende + vollständige Adresse  bzw',
 'de/mitglied werden  Oder Sie unte  für ein 1olo)uan] Sjelzyote[c} Spen steuerlich absetzbar',
 'Indem Sie Mitmachen:  afd',
 'Wie',
 '170    SEICCHRSE unsere Grundrechte',
 ', 147,148, 149, 150, 151, 152, 153, 154, 165, 176,',
 ', 25, 41,42, 43, 44, 45, 46, 55, 56, 71,  , 176,180,',
 ',52  , 68,131    ,',
 ',79  ,198, 199, 200, 201, 202, 204, 205,',
 '43, 45, 46, 165,',
 '126, 127,131  , 98, 144,185      , 77,101  , 34, 35, 36, 37,38, 39, 51,106, 130,',
 ', 68, 139, 140, 141,199    ,129  ,154  ,12, 13, 14, 15, 16, 17, 18, 19, 23,',
 ', 13, 14, 15, 16, 17, 28, 45,',
 ',120  , 178,194  , 152,',
 ', 165,181  , 63, 78, 79, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, , 101,150,',
 '158      ,',
 '197,198, 199, 200, 

In [7]:
import numpy as np
import json
import glob

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lang.de.examples import sentences 
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import pickle 

import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

/opt/anaconda3/envs/NLP_spacy/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [84]:
german_stop_words= stopwords.words('german')

In [8]:
spacy.info("de_core_news_lg")

{'lang': 'de',
 'name': 'core_news_lg',
 'version': '3.4.0',
 'description': 'German pipeline optimized for CPU. Components: tok2vec, tagger, morphologizer, parser, lemmatizer (trainable_lemmatizer), senter, ner.',
 'author': 'Explosion',
 'email': 'contact@explosion.ai',
 'url': 'https://explosion.ai',
 'license': 'MIT',
 'spacy_version': '>=3.4.0,<3.5.0',
 'spacy_git_version': 'dd038b536',
 'vectors': {'width': 300,
  'vectors': 500000,
  'keys': 500000,
  'name': 'de_vectors'},
 'labels': {'tok2vec': [],
  'tagger': ['$(',
   '$,',
   '$.',
   'ADJA',
   'ADJD',
   'ADV',
   'APPO',
   'APPR',
   'APPRART',
   'APZR',
   'ART',
   'CARD',
   'FM',
   'ITJ',
   'KOKOM',
   'KON',
   'KOUI',
   'KOUS',
   'NE',
   'NN',
   'NNE',
   'PDAT',
   'PDS',
   'PIAT',
   'PIS',
   'PPER',
   'PPOSAT',
   'PPOSS',
   'PRELAT',
   'PRELS',
   'PRF',
   'PROAV',
   'PTKA',
   'PTKANT',
   'PTKNEG',
   'PTKVZ',
   'PTKZU',
   'PWAT',
   'PWAV',
   'PWS',
   'TRUNC',
   'VAFIN',
   'VAIMP',
   'V

In [9]:
def lemmatization_test(texts,  allowed_posttags=['NOUN','ADJ','VERB','ADV']):
    nlp=spacy.load('de_core_news_lg',disable=['parser','ner'])
    texts_out=[]
    for text in tqdm(texts):
        doc= nlp(text)
        new_text=[]
        for token in doc:
            if token.pos_ in allowed_posttags:
                new_text.append(token.lemma_)
        final=' '.join(new_text)
        texts_out.append(final)
    return texts_out

In [26]:
lemma_text=lemmatization_test(text)

100%|██████████| 2086/2086 [00:04<00:00, 460.31it/s]


In [44]:
lemma_text = list(filter(None, lemma_text))

In [45]:
lemma_text[::-1]

['spenden Paypal@alternativefuer',
 'Afd',
 'AfD-ID',
 'Bankverbindung Alternative Berliner IBAN DE94 1(6 % bitte geben Betreff Spende vollständig Adresse',
 'Mitglied unen Spen steuerlich absetzbar',
 'Mitmachen',
 'wie',
 'Grundrecht',
 '71,148',
 'Parallelgesellschaft Partei PCR-Test Pflege Photovoltaikanlage Polizei Quote Rechtsstaat Rente Rundfunk Schuld Seenotrettung Souveränität Sozialhilfe Sozialismus Staatsangehörigkeit Steuer Subvention Terrorismus Umverteilung UN-Flüchtlingspakt UN-Migrationspakt Verbraucherschutz Verschuldung Wehrpflicht Wirtschaft Wissenschaft Wohngeld Index',
 'Klima Klimawandel Kriminalität Kultur Landwirtschaft Leitkultur Linksextremismus Linksterrorismus Lobbyismus Meinungsfreiheit Migration Mindestlohn Mittelstand Multikulturalismus Muslimbruderschaft Stream',
 '87,158',
 '99,100',
 '85,108',
 '80,118',
 '99,100',
 'Droge Eigentum Einwanderung Energie Energieträger Enteignung Entwicklungshilfe Erbschaftssteuer Familie Flüchtling Gefährder Gesundheitss

In [46]:
def gen_words(texts):
    final=[]
    for text in tqdm(texts):
        new= gensim.utils.simple_preprocess(text,deacc=True)
        final.append(new)
    return final

In [47]:
data_words=gen_words(lemma_text)

100%|██████████| 1901/1901 [00:00<00:00, 41764.86it/s]


In [48]:
bigram_phrases=gensim.models.Phrases(data_words,min_count=3,threshold=100)
trigram_phases=gensim.models.Phrases(bigram_phrases[data_words],threshold=50)

bigram=gensim.models.phrases.Phraser(bigram_phrases)
trigram=gensim.models.phrases.Phraser(trigram_phases)

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trgram(texts):
    return [trigram[bigram[doc]] for doc in texts]

data_bigrams=make_bigrams(data_words)
data_bigrams_trigrams=make_trgram(data_bigrams)

In [49]:
data_bigrams_trigrams

[['normal'],
 ['normal'],
 ['programm', 'alternative', 'wahl'],
 ['deutsch', 'bundestag'],
 ['demokratie_rechtsstaat'],
 ['volk', 'souveran'],
 ['volksabstimmung', 'schweizer_modell'],
 ['gewaltenteilung', 'gewahrleisten'],
 ['justiz'],
 ['reformieren'],
 ['trennung', 'amt', 'mandat', 'kampf', 'amterpatronage'],
 ['macht', 'partei', 'beschranken'],
 ['frei', 'listenwahl', 'frei', 'mandat'],
 ['verkleinerung', 'parlament'],
 ['amtszeit', 'begrenzen'],
 ['lobbyismus', 'eindammen'],
 ['freiheit', 'verantwortung'],
 ['burgerlich', 'freiheitliche', 'rechtsordnung'],
 ['leistung', 'quote', 'freiheit', 'diskriminierung'],
 ['schutz', 'personlich', 'freiheit', 'digital', 'ubergriffen'],
 ['hinweisgeber', 'schutzen'],
 ['vaterland'],
 ['zuruckfuhrung', 'europaisch_union', 'staatenbund', 'souveran', 'staat'],
 ['steuer_finanz'],
 ['eu', 'steuern'],
 ['steuersystem', 'steuerart', 'verstandlich', 'systematik'],
 ['abschaffung', 'substanzsteuer'],
 ['land'],
 ['elementar', 'steuerburger'],
 ['beste

In [50]:
from gensim.models import TfidfModel

id2word=corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus=[id2word.doc2bow(text) for text in texts]

print(corpus[3])

tfidf=TfidfModel(corpus,id2word=id2word)

low_value=0.03
words=[]
words_missing_in_tfdf=[]

for i in tqdm(range(0,len(corpus))):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops=low_value_words+words_missing_in_tfdf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]  
    corpus[i]=new_bow


[(4, 1), (5, 1)]


100%|██████████| 1901/1901 [00:00<00:00, 37706.29it/s]


In [51]:
topics=['climate',
'economy',
'education',
'health',
'infrastructure',
'science',
'social causes',
'politics and ideology',
'technology and entrepreneurship']

In [52]:
num_topics=len(topics)

In [53]:
lda_model=gensim.models.ldamodel.LdaModel(
corpus=corpus,
id2word=id2word,
num_topics=num_topics,
random_state=100,
update_every=1,
chunksize=100,
passes=10,
alpha='auto'
)

In [54]:
dic={}
dic_sum={}
for idx, topic in lda_model.print_topics(num_topics,num_words=100):
    elements=[]
    percent=[]

    for e in topic.split('+'):
        elements.append(e.split('*')[1].replace('"','').strip())
        percent.append(float(e.split('*')[0].replace('"','').strip()))

    dic[str(idx)+'_word']=elements
    dic[str(idx)+'_per']=percent
    dic_sum[idx]=sum(percent)
print(dic_sum)

{0: 0.5030000000000003, 1: 0.47600000000000026, 2: 0.49400000000000027, 3: 0.4700000000000003, 4: 0.5220000000000004, 5: 0.4940000000000003, 6: 0.6790000000000003, 7: 0.49500000000000033, 8: 0.5420000000000004}


In [55]:
pd.DataFrame(data=dic)

,0_word,0_per,1_word,1_per,2_word,2_per,3_word,3_per,4_word,4_per,5_word,5_per,6_word,6_per,7_word,7_per,8_word,8_per
0,deutsch,0.057,offentlich,0.026,wieder,0.020,landwirtschaft,0.025,fordern,0.074,setzen,0.032,paypal,0.115,ausbau,0.020,auch,0.063
1,gut,0.027,erhalten,0.023,dazu,0.015,regional,0.022,nur,0.030,ein,0.025,alternativefuer,0.115,digital,0.020,lehnen,0.033
2,kultur,0.018,frei,0.016,insbesondere,0.013,mensch,0.018,familie,0.020,deshalb,0.015,spenden,0.115,eu,0.017,hoch,0.019
3,sprache,0.014,neu,0.013,ermoglichen,0.013,europaisch,0.015,bleiben,0.017,wald,0.012,burger,0.020,versorgung,0.017,fuhren,0.017
4,schule,0.013,nutzung,0.012,landlich,0.013,starken,0.015,daher,0.016,dafur,0.011,land,0.013,klima_energie_technik_digitalisierung,0.016,sozial,0.017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,stellung,0.002,bitte,0.002,bzw,0.002,konventionell,0.002,last,0.002,gesondert,0.002,ungerechtigkeit,0.001,anteil,0.002,wie,0.002
96,kulturgut,0.002,iban,0.002,standig,0.002,islam,0.002,beteiligter,0.002,gesicherte,0.002,app,0.001,fachen,0.002,zugute,0.002
97,rentensystem,0.002,folgend,0.002,lebensabend,0.002,entweren,0.002,festgelegt,0.002,zensur,0.002,endlich,0.001,erwarmung,0.002,derart,0.002
98,festschreiben,0.002,abgabenlast,0.002,rentenalter,0.002,aufheizung,0.002,wirken,0.002,werbung,0.002,grundsicherung,0.001,emissione,0.002,zugang,0.002


In [56]:
pyLDAvis.enable_notebook()
vis=gensimvis.prepare(lda_model,corpus,id2word,mds='mmds',R=30)
vis

/opt/anaconda3/envs/NLP_spacy/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/opt/anaconda3/envs/NLP_spacy/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.100266  0.351095       1        1  16.098735
8      0.294055  0.092383       2        1  14.714888
4     -0.241257  0.105460       3        1  13.289037
5      0.057784 -0.272712       4        1  10.686051
7      0.099218  0.185349       5        1  10.054408
0     -0.149893 -0.223674       6        1   9.554732
1     -0.210116 -0.066538       7        1   8.970947
2      0.034076 -0.034137       8        1   8.434545
3      0.216400 -0.137227       9        1   8.196657, topic_info=                 Term        Freq       Total Category  logprob  loglift
4708          spenden  249.000000  249.000000  Default  30.0000  30.0000
4706  alternativefuer  249.000000  249.000000  Default  29.0000  29.0000
4707           paypal  249.000000  249.000000  Default  28.0000  28.0000
173           fordern  132.000000  132.000000  Default  27.0000  27.0000
485              auch  124.000000  124.000000  Default  26.0000  26.0000
...               ...         ...         ...      ...      ...      ...
381         erfullung    5.485432    6.257392   Topic9  -5.2993   2.3698
4329   verbotspolitik    5.355748    6.135932   Topic9  -5.3232   2.3655
318         forschung    5.091397    5.860213   Topic9  -5.3739   2.3608
2346           person    5.025875    5.794436   Topic9  -5.3868   2.3591
472         kulturell    4.925560    5.694081   Topic9  -5.4070   2.3565

[309 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
461       1  0.956499          aber
57        8  0.970986   abschaffung
155       7  0.858421   abschiebung
1281      3  0.971486           afd
390       5  0.912756  agrarpolitik
...     ...       ...           ...
515       4  0.933129        zahlen
1101      8  0.971504       zentral
884       5  0.976202          ziel
1151      9  0.969994         zudem
1490      5  0.956571     zunehmend

[284 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 9, 5, 6, 8, 1, 2, 3, 4])